In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans



In [2]:
df = pd.read_csv(r"C:\Users\ZAK-TECH\Desktop\KAIM_week4\data\raw\data (1).csv")

In [4]:
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'])
snapshot_date = df['TransactionStartTime'].max() + pd.Timedelta(days=1)

rfm = df.groupby('CustomerId').agg({
    'TransactionStartTime': lambda x: (snapshot_date - x.max()).days,
    'TransactionId': 'count',
    'Amount': 'sum'
}).reset_index()

rfm.rename(columns={
    'TransactionStartTime': 'Recency',
    'TransactionId': 'Frequency',
    'Amount': 'Monetary'
}, inplace=True)

rfm_features = ['Recency', 'Frequency', 'Monetary']
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[rfm_features])

kmeans = KMeans(n_clusters=3, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

cluster_summary = rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean()
high_risk_cluster = cluster_summary.sort_values(
    by=['Recency', 'Frequency', 'Monetary'],
    ascending=[False, True, True]
).index[0]

rfm['is_high_risk'] = (rfm['Cluster'] == high_risk_cluster).astype(int)
df = df.merge(rfm[['CustomerId', 'is_high_risk']], on='CustomerId', how='left')
print(df[['CustomerId', 'is_high_risk']].drop_duplicates().head())

        CustomerId  is_high_risk
0  CustomerId_4406             0
2  CustomerId_4683             1
3   CustomerId_988             0
5  CustomerId_1432             1
6  CustomerId_2858             0
